## Importing the required libraries

In [9]:
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer # type: ignore
from sklearn.metrics.pairwise import cosine_similarity
import nltk

## Reading the data and preprocessing it

In [10]:
data = pd.read_csv('data/mental_feed.csv')
data = pd.read_csv('data/mental_feed.csv')

# Function to tokenize sentences and words
def tokenize_sentence(text):
    sentence_tokens = nltk.sent_tokenize(text)
    return sentence_tokens
def tokenize_word(text):
    word_tokens = nltk.word_tokenize(text)
    return word_tokens

# Apply tokenization to each question and answer
data['Question_Sentence_Tokens'] = data['question'].apply(tokenize_sentence)
data['Question_Word_Tokens'] = data['question'].apply(tokenize_word)
data['Answer_Sentence_Tokens'] = data['answer'].apply(tokenize_sentence)
data['Answer_Word_Tokens'] = data['answer'].apply(tokenize_word)

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['question'])

## Function which actually generates the reply for chatbot
> Checks the intent

> Tokenises user input

> Vectorises it

> Replies according to either intent or the datset provided

In [11]:
def hybrid_chatbot(user_input):
    patterns = {
        "greet": r"hello|hi|hey",
        "thanks": r"thank you|thanks"
    }
    responses = {
        "greet": "Hello! How can I help you today?",
        "thanks": "You're welcome!"
    }
    
    # Checking for pattern matches first
    for intent, pattern in patterns.items():
        if re.search(pattern, user_input, re.IGNORECASE):
            return responses[intent]
    
    # Tokenize user input
    user_input_sentence_tokens = tokenize_sentence(user_input) 
    user_input_word_tokens = tokenize_word(user_input)
    user_input_str = ' '.join(user_input_word_tokens)
    user_input_vector = vectorizer.transform([user_input_str])
    # Using the cosine similarity and using the most similar response to return output
    similarities = cosine_similarity(user_input_vector, X).flatten()
    response = data.iloc[similarities.argmax()]['answer']
    return response

### Test the chatbot
To exit the chat, reply with bye, goodbye, end, quit, exit or terminate

In [12]:
print("=====>Chatbot: Hello there! Tell me how you feel and I will try to help you with it")
while True:
    user_input = input("=====>You: ")
    if re.search(r"bye|goodbye|end|quit|exit", user_input):
        print("=====>Chatbot: Goodbye, take care.")
        break
    response = hybrid_chatbot(user_input)
    print("=====>Chatbot:", response)

=====>Chatbot: Hello there! Tell me how you feel and I will try to help you with it
=====>Chatbot: It sounds like you might be experiencing symptoms of depression. Common signs of depression include persistent sadness, loss of interest in activities, changes in appetite and weight, sleep disturbances, fatigue, difficulty concentrating, and thoughts of death or suicide. It's important to seek help from a mental health professional for an accurate diagnosis and appropriate treatment.
=====>Chatbot: It sounds like you might be experiencing symptoms of depression. Common signs of depression include persistent sadness, loss of interest in activities, changes in appetite and weight, sleep disturbances, fatigue, difficulty concentrating, and thoughts of death or suicide. It's important to seek help from a mental health professional for an accurate diagnosis and appropriate treatment.
=====>Chatbot: Goodbye, take care.
